# **CONTROLLING METFORMIN**

<img src="https://raw.github.com/joseph-hellerstein/advanced-controls-lectures/main/lectures/images/simple_feedback_nofilter.png"
     alt="Markdown Monster icon" 
     width="600" height="750"
     style="float: left; margin-right: 10px;" />

This notebook uses ``controlSBML`` to construct an PID controller for a Metformin producing pathway.

<img src="https://raw.github.com/ModelEngineering/controlSBML/main/examples/images/metformin_overview.png"
     alt="Markdown Monster icon" 
     width="800" height="1000"
     style="float: left; margin-right: 10px;" />

In [2]:
import numpy as np
try:
    import controlSBML as ctl
except:
    !pip install controlSBML
    import controlSBML as ctl
try:
    import control
except:
    !pip install control
    import control
#
from controlSBML import ControlSBML, constants
from controlSBML.grid import Grid
import matplotlib.pyplot as plt
from typing import Optional, List

# Helpers

In [3]:
s = control.TransferFunction.s
TIMES = np.linspace(0, 5, 500)
WOLF_URL = constants.WOLF_URL

## plotStep

In [4]:
def plotStep(tf, title:str="", times=TIMES, xlim:Optional[list]=None, figsize=[5,5],
      is_plot=True):
    """
    Plots the step response of the transfer function.

    Args:
        tf - transfer function
    """
    _, ax = plt.subplots(1, 1, figsize=figsize)
    _, yv = control.step_response(tf, T=times)
    _ = ax.plot(times, yv)
    if xlim is not None:
        ax.set_xlim(xlim)
    _ = ax.set_title(title)
    if not is_plot:
        plt.close()

# TESTS
tf = control.TransferFunction([5], [1, 5])
plotStep(tf, is_plot=False, xlim=[0, 3])
print("OK!")

OK!


## complex_magnitude

In [5]:
def complex_magnitude(z):
    return (z.real**2 + z.imag**2)**0.5

## ppComplex

In [6]:
def ppComplex(complexes:np.ndarray, round_digits=3)->str:
    """
    Constructs a pretty print representation of a complex number.
    """
    complexes = np.array(complexes)
    complexes = complexes.flatten()
    if not isinstance(complexes, list):
        complexes = [complexes]
    results = []
    if isinstance(complexes[0], np.ndarray):
        complexes = complexes[0]
    for cmpx in complexes:
        try:
            if np.imag(cmpx) == 0:
                results.append(str(round(np.real(cmpx), round_digits)))
            else:
                 results.append(str(round(np.real(cmpx), round_digits)) + "+" + str(round(np.imag(cmpx), round_digits)) + "j")
        except:
            import pdb; pdb.set_trace()
    return ", ".join(results)

# Tests
result = ppComplex((-1.9999999999999998+0j))
result = ppComplex([3+2j, 4])
assert("j" in result)
print("OK!")

OK!


## plotRootLocusWithGains

In [7]:
def plotRootLocusWithGains(open_loop_transfer_function:control.TransferFunction,
      gains:List[float],
      xlim:Optional[np.ndarray]=None,
      title:Optional[str]=None,
      ylim:Optional[np.ndarray]=None,
      is_annotate:bool=True,
      markersize_multiplier:float=100,
      figsize=(5, 5),
      ax=None,
      is_plot:bool=True)->plt.axes:
    """
    Constructs a root locus plot with red hexagons for closed loop poles at different gains.
    Poles at specific gains are indicated by hexigon markers.

    Args:
        open_loop_transfer_function
        gains
        xlim: limits on the axis
        size_multiplier: multiplies by gain to get size of hexagon that marks the pole
        figsize: width and height of figure
        is_annotate: Annotate gain hexigons with gain values
    """
    if ax is None:
        _, ax = plt.subplots(1, 1, figsize=figsize)
    # Plot the root locus
    _ = control.root_locus(open_loop_transfer_function, grid=False, plot=True, xlim=xlim,
          ylim=ylim, ax=ax)
    # Construct the title
    if title is None:
        poles = open_loop_transfer_function.poles()
        zeros = open_loop_transfer_function.zeros()
        title = ""
        if len(poles) > 0:
            title += f"FFL poles: {ppComplex(poles)}  "
        if len(zeros) > 0:
            title += f"FFL zeros: {ppComplex(zeros)}"
    ax.set_title(title)
    # Add points for closed loop poles
    for gain in gains:
        closed_loop_transfer_function = control.feedback(gain*open_loop_transfer_function)
        poles = closed_loop_transfer_function.poles()
        xv = []
        yv = []
        for pole in poles:
            xv.append(pole.real)
            yv.append(pole.imag)
        if is_annotate:
            annotation = str(gain)
        else:
            annotation = ""
        ax.scatter(xv, yv, s=markersize_multiplier*gain, marker="h", color="r")
        [ ax.annotate(annotation, (xv[i], yv[i]), color='blue', rotation=25)
             for i in range(len(poles))]
        arrowprops=dict(facecolor='black', shrink=0.05)
    # Plot at zero
    ax.plot([0, 0], [-10, 10], color="grey", linestyle="--")
    #
    if not is_plot:
        plt.close()
        ax = None
    return ax

# Tests
tf = control.zpk([1], [-1, -2], [1])
plotRootLocusWithGains(tf, [0.2, 0.5], markersize_multiplier=200, ylim=[-3, 3], xlim=[-3.5, 0], is_annotate=True,
      figsize=(2, 2), is_plot=False)
print("OK!")

OK!


# Step 1. State design objectives (5 pt)

1. Select a model and load it into Tellurium. Print it in Antimony.
1. Specify the output from the model that you want to control.
2. One of your design objectives is that the closed loop system is stable. Provide two additional objectives such as: eliminate bias (the system converges to the desired output); no oscillations; settling times no greater than X (where you specify X); no overshoot (undershoot).


# Step 2. Find the control input and operating region (5 pt)

1. Choose a suitable input to control the system output. Plot a staircase function of the input over the operating region. Specify:
      1. directional effect of the input on the output
      1. operating region for the input
      1. range of outputs that can be achieved (feasible setpoints)
1. Write a short narrative describing how the plot indicates that the control objectives can be achieved over the operating region. 

# Step 3. Do system identification (5 pt)

For the system and operating point that you selected, find a transfer function that fits the staircase response.
By *fit*, I mean that the transfer function broadly follows the predicted value.
If you later find that your theorty-based design (step 4) provides little insight into your final design (step 5), you may need to revisit this step.

# Step 4. Construct a preliminary, theory-based design (15 pt)

Use the transfer function in step 3 to do a theory-based design using the root locus plot to do PID control.
Start by answering the following questions:
1. Given the open loop poles and zeros of your system, what is the best settling time you can expect? (Recall that root locus branches start at poles and terminate at zeros.)
2. How does your answer change if you use I-control (e.g., PI, PID)?

Write a short narrative that describes the logic you used in your theory-based design.

# Step 5. Construct a final design using a testbed (15 pt)

1. The theory-based design provides a starting point for your final design that is based on a test-bed as you did in Homework 1. Use intutitions from the root locus plot to adjust the values of design parameters.
2. Once you are satisfied with your final design, revisit system identification and theory-based design to see if a different number of poles and/or zeroes in your system identification provides a better explanation of your final design.

# Step 6. Evaluate the final design (5 pt)

Answer the following questions:
1. Did you need to change your system objectives? At what points did you did you do this?
2. What criteria did you use to determine that your system identification was sufficient?
3. How close was your theory-based design to the testbed-based design that you chose.